난독화된 한글 리뷰 복원 및 생성 AI 경진대회 </br>
+ 플러스 알파 : (나쁨)0~5(좋음)점에서 몇 점에 해당하는지 판단할 수 있다면?

Basline Code

단어 사전 기반 복원과 LLM(Gemma-7b)을 활용한 텍스트 변환 및 추론

In [1]:
# Data Load
import pandas as pd

train = pd.read_csv('./data/train.csv', encoding='utf-8-sig')
test = pd.read_csv('./data/test.csv', encoding='utf-8-sig')

In [2]:
train.head(10)

,ID,input,output
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...
5,TRAIN_00005,펀냔휜 잘 쉭곶 왔쑵닝따. 준윙에 맏쥡됴 만학썩 좋흖 겼 갇따용.,편안히 잘 쉬고 왔습니다. 주위에 맛집도 많아서 좋은 것 같아요.
6,TRAIN_00006,"쓰윔튿룸위 깝써칙갖 있네 ㅋ 꼭굽 췸규웨 낑펙투, 합뻔퓨 옥쪽, 캬뻬 깥은 띰 떼잎...","스위트룸의 값어치가 있네 ㅋ 고급 침구에 킹베드, 하버뷰 욕조, 카페 같은 티 테이..."
7,TRAIN_00007,쨉빵뮨 의싸 쩐허 없쓺. 씬랏슬톄잎뾰댜 떠 짝음. 홈뗄 줄찾짱 15땜만 캉눙. 혹텔...,재방문 의사 전혀 없음. 신라스테이보다 더 작음. 호텔 주차장 15대만 가능. 호텔...
8,TRAIN_00008,념묵 멎쥑교 꽁귀 좋습니닸. 췐곡윕뉘댜!,너무 멋지고 공기 좋습니다. 최고입니다!
9,TRAIN_00009,"짱졈: 쩡켤함, 츄챠 씨셜 죠흠, 쉼섦 좋음. 단젊: 줏짢 츌짜할 떼 뷸뻔함, ...","장점: 청결함, 주차 시설 좋음, 시설 좋음. 단점: 주차 출차할 때 불편함, ..."


In [3]:
# 단어 사전 생성

match_dict = {}

for input_text, output_text in zip(train['input'], train['output']):
    input_words = input_text.split()
    output_words = output_text.split()
    for iw, ow in zip(input_words, output_words):
        match_dict[iw] = ow


In [4]:
# 변환 적용

def replace_words(input_text, match_dict):
    words = input_text.split()
    replaced_words = [match_dict.get(word, word) for word in words]
    return " ".join(replaced_words)

In [5]:
converted_reviews = test['input'].apply(lambda x:replace_words(x, match_dict)).tolist()

In [7]:
# Submission
submission = pd.read_csv('./data/sample_submission.csv', encoding='utf-8-sig')
submission['output'] = converted_reviews
submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')

LLM활용 (Gemma)

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [ ]:
# Data Load

train = pd.read_csv('./data/train.csv', encoding='utf-8-sig')
test = pd.read_csv('./data/test.csv', encoding='utf-8-sig')

In [ ]:
# Model Load

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = 'beomi/gemma-ko-7b'
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [ ]:
# Inference

pipe = pipeline(
    task = "text-generation",
    model=model,
    tokenizer=tokenizer
)

restored_reviews = []

for index, row in test.iterrows():
    query = row['input']

    messages = [
         {
            "role": "system",
            "content": (
                "You are a helpful assistant specializing in restoring obfuscated Korean reviews. "
                "Your task is to transform the given obfuscated Korean review into a clear, correct, "
                "and natural-sounding Korean review that reflects its original meaning. "
                "Below are examples of obfuscated Korean reviews and their restored forms:\n\n"
                f"Example, {samples}"  
                "Spacing and word length in the output must be restored to the same as in the input. "
                "Do not provide any description. Print only in Korean."
            )
        },
        {
            "role": "user",
            "content": f"input : {query}, output : "
        },
    ]

    prompt = "\n".join([m["content"] for m in messages]).strip()
        

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
        max_new_tokens=len(query),
        eos_token_id=pipe.tokenizer.eos_token_id
    )

    generated_text = outputs[0]['generated_text']
    result = generated_text[len(prompt):].strip()
        

    restored_reviews.append(result)

In [ ]:
# Submission
submission = pd.read_csv('./sample_submission.csv', encoding='utf-8-sig')
submission['output'] = restored_reviews
submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd

train = pd.read_csv('./train.csv', encoding='utf-8-sig')

In [ ]:
import pandas as pd

HangeulFile = pd.read_csv('./data/train.csv')
HangeulFile.head(20)

,ID,input,output
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...
5,TRAIN_00005,펀냔휜 잘 쉭곶 왔쑵닝따. 준윙에 맏쥡됴 만학썩 좋흖 겼 갇따용.,편안히 잘 쉬고 왔습니다. 주위에 맛집도 많아서 좋은 것 같아요.
6,TRAIN_00006,"쓰윔튿룸위 깝써칙갖 있네 ㅋ 꼭굽 췸규웨 낑펙투, 합뻔퓨 옥쪽, 캬뻬 깥은 띰 떼잎...","스위트룸의 값어치가 있네 ㅋ 고급 침구에 킹베드, 하버뷰 욕조, 카페 같은 티 테이..."
7,TRAIN_00007,쨉빵뮨 의싸 쩐허 없쓺. 씬랏슬톄잎뾰댜 떠 짝음. 홈뗄 줄찾짱 15땜만 캉눙. 혹텔...,재방문 의사 전혀 없음. 신라스테이보다 더 작음. 호텔 주차장 15대만 가능. 호텔...
8,TRAIN_00008,념묵 멎쥑교 꽁귀 좋습니닸. 췐곡윕뉘댜!,너무 멋지고 공기 좋습니다. 최고입니다!
9,TRAIN_00009,"짱졈: 쩡켤함, 츄챠 씨셜 죠흠, 쉼섦 좋음. 단젊: 줏짢 츌짜할 떼 뷸뻔함, ...","장점: 청결함, 주차 시설 좋음, 시설 좋음. 단점: 주차 출차할 때 불편함, ..."


문장 길이 체크

In [ ]:
def checkSentenceLen():
    input_length = HangeulFile['input'].str.len()
    output_length  = HangeulFile['output'].str.len()
    HangeulFile['isSameLen'] = input_length == output_length
    print('문장 길이 체크\n', HangeulFile['isSameLen'].value_counts())

checkSentenceLen()

문장 길이 체크
 isSameLen
True     10935
False      328
Name: count, dtype: int64


In [ ]:
HangeulFile[HangeulFile['isSameLen'] == False]

# input_length+1 => output_length
# ex) ID = TRAIN_00069
input_text = HangeulFile[HangeulFile['ID'] == 'TRAIN_00069']['input']
output_text = HangeulFile[HangeulFile['ID'] == 'TRAIN_00069']['output']
# output_text 뒤에 공백 존재
print(list(input_text))
print(list(output_text))

# 공백 제거
HangeulFile['output'] = HangeulFile['output'].str.rstrip()
checkSentenceLen()

['쩡결학꼬 찐절하쉬넥오']
['청결하고 친절하시네요 ']
문장 길이 체크
 isSameLen
True    11263
Name: count, dtype: int64


특징

0. 문장 공백 유지 </br> 
1. 리뷰데이터이기 때문에 띄어쓰기는 따로 처리 x</br>
2. ㄱ -> ㅋ, ㄱ -> ㄲ 로 쓰는 경우가 많음</br>
    a. 된소리, 거센소리를 평음으로 바꾸는 작업이 필요</br>
3. 받침을 의도적으로 넣는 경우가 많음 </br>
    a. 받침을 의도적으로 넣은 경우 삭제가 필요함</br>
    * 아직 원래 있었던 받침인지 아닌지를 모르니 일단 다 삭제해보자</br>
    b. 대신 필요한 받침인지 아닌지 구분이 필요함</br>
4. 받침을 대체하는 것도 많음. 응대 -> 읗뎃</br>
    a. 대체된 받침인지도 구분할 줄 알아야 함</br>
5. 대 -> 데 아이 어이도 바꿈</br>
6. 종성이 랉, 홅, 섧 같은 자음의 조합으로도 나타남

된소리되기 현상</br>
깍두기 -> 깍뚜기</br>

사이시옷 현상</br>
심고있다 -> 심꼬있다</br>

형태소 분석

In [ ]:
# 한국어 형태소 분석 라이브러리
# Okt : Open Korea text
# Mecab, Komoran, Hannanum, Kkma

from konlpy.tag import Okt

okt = Okt()

text = HangeulFile[HangeulFile['ID'] == 'TRAIN_00000']['input'].iloc[0]
words = text.split()

# 문장 공백 유지
tokens = []
for word in words:
    tokens.extend(okt.morphs(word))
    tokens.append(' ')

if tokens and tokens[-1] == ' ':
    tokens = tokens[:-1]

print(tokens)


['별', ' ', '한', ' ', '게토', ' ', '았깝땀', '.', ' ', '왜', ' ', '싸람', '듯릭', ' ', '펼', ' ', '1', '캐', '를', ' ', '쥰눈징', ' ', '컥꺾폰', ' ', '싸람', '믐롯섞', ' ', '맒록', ' ', '섧멍핥쟈닐', ' ', '탯끎룐눈', ' ', '녀뮤', ' ', '퀼교', '...', ' ', '야뭍툰', ' ', '둠', ' ', '변', ' ', '닺씨', ' ', '깍', '낄', ' ', '싫훈', ' ', '굣', '.', ' ', '깸삥읊', ' ', '20', '여', ' ', '년', ' ', '댜녁뵨', ' ', '곧', ' ', '중', ' ', '쩨윌', ' ', '귑푼', ' ', '낙', '팠', '떤', ' ', '곶', '.']


거센소리, 된소리 -> 평음 맵핑

In [ ]:
# 예사소리(평음) : ㄱ, ㄷ, ㅂ, ㅅ, ㅈ
# 된소리 : ㄲ, ㄸ, ㅃ, ㅆ, ㅉ
# 거센소리 : ㅋ, ㅌ, ㅍ, ㅊ
aspirated_consonants = {'ㅋ':'ㄱ', 'ㅌ':'ㄷ', 'ㅍ':'ㅂ', 'ㅊ':'ㅅ', 'ㅊ':'ㅈ'}
strengthened_consonants = {'ㄲ':'ㄱ', 'ㄸ':'ㄷ', 'ㅃ':'ㅂ', 'ㅆ':'ㅅ', 'ㅉ':'ㅈ'}

def to_plainWord(x):
    # 초성 변환
    if x in strengthened_consonants:
        x = strengthened_consonants[x]
    elif x in aspirated_consonants:
        x = aspirated_consonants[x]

    # 종성 변환 (예: ㅆ, ㅉ 등이 종성으로 올 경우)
    if x in strengthened_consonants:
        x = strengthened_consonants[x]
    elif x in aspirated_consonants:
        x = aspirated_consonants[x]

    return x



(일단) 받침 제거</br>
빳닻갉 -> 바닷가 어떻게??</br>
밧닷갉 -> 바다가 초성,중성 평음화 + 받침삭제하면 이렇게 되어버림</br>

** 문맥 기반 언어 모델 활용</br>
편지를 밝았다 x</br>
편지를 받았다 o</br>

In [ ]:
##

tokens는 조합된 음절 </br>
1. 자음, 모음 분리 </br>
2. 된소리, 거센소리 -> 평음으로 치환 </br>
3. 다시 조합</br>

대표적인 라이브러리 : hgtk, jamo

In [ ]:
import hgtk

sentence = []
for token in tokens:
    chars = [hgtk.letter.decompose(c) if hgtk.checker.is_hangul(c) else c for c in token]
    print(chars)
    normalized_chars = []
    for char in chars:       
        if isinstance(char, tuple) and len(char) == 3:
            cho, jung, jong = char

            # 초성, 종성 변환
            cho = to_plainWord(cho)
            jong = to_plainWord(jong)
            
            normalized_char = hgtk.letter.compose(cho, jung, jong)
            normalized_chars.append(normalized_char)
        else:
            # 한글이 아니면 그대로 추가
            normalized_chars.append(char)

    sentence.append(''.join(normalized_chars))

print(''.join(sentence))
        

[('ㅂ', 'ㅕ', 'ㄹ')]
[' ']
[('ㅎ', 'ㅏ', 'ㄴ')]
[' ']
[('ㄱ', 'ㅔ', ''), ('ㅌ', 'ㅗ', '')]
[' ']
[('ㅇ', 'ㅏ', 'ㅆ'), ('ㄲ', 'ㅏ', 'ㅂ'), ('ㄸ', 'ㅏ', 'ㅁ')]
['.']
[' ']
[('ㅇ', 'ㅙ', '')]
[' ']
[('ㅆ', 'ㅏ', ''), ('ㄹ', 'ㅏ', 'ㅁ')]
[('ㄷ', 'ㅡ', 'ㅅ'), ('ㄹ', 'ㅣ', 'ㄱ')]
[' ']
[('ㅍ', 'ㅕ', 'ㄹ')]
[' ']
['1']
[('ㅋ', 'ㅐ', '')]
[('ㄹ', 'ㅡ', 'ㄹ')]
[' ']
[('ㅈ', 'ㅠ', 'ㄴ'), ('ㄴ', 'ㅜ', 'ㄴ'), ('ㅈ', 'ㅣ', 'ㅇ')]
[' ']
[('ㅋ', 'ㅓ', 'ㄱ'), ('ㄲ', 'ㅓ', 'ㄲ'), ('ㅍ', 'ㅗ', 'ㄴ')]
[' ']
[('ㅆ', 'ㅏ', ''), ('ㄹ', 'ㅏ', 'ㅁ')]
[('ㅁ', 'ㅡ', 'ㅁ'), ('ㄹ', 'ㅗ', 'ㅅ'), ('ㅅ', 'ㅓ', 'ㄲ')]
[' ']
[('ㅁ', 'ㅏ', 'ㄻ'), ('ㄹ', 'ㅗ', 'ㄱ')]
[' ']
[('ㅅ', 'ㅓ', 'ㄼ'), ('ㅁ', 'ㅓ', 'ㅇ'), ('ㅎ', 'ㅏ', 'ㄾ'), ('ㅈ', 'ㅑ', ''), ('ㄴ', 'ㅣ', 'ㄹ')]
[' ']
[('ㅌ', 'ㅐ', 'ㅅ'), ('ㄲ', 'ㅡ', 'ㄻ'), ('ㄹ', 'ㅛ', 'ㄴ'), ('ㄴ', 'ㅜ', 'ㄴ')]
[' ']
[('ㄴ', 'ㅕ', ''), ('ㅁ', 'ㅠ', '')]
[' ']
[('ㅋ', 'ㅟ', 'ㄹ'), ('ㄱ', 'ㅛ', '')]
['.', '.', '.']
[' ']
[('ㅇ', 'ㅑ', ''), ('ㅁ', 'ㅜ', 'ㅌ'), ('ㅌ', 'ㅜ', 'ㄴ')]
[' ']
[('ㄷ', 'ㅜ', 'ㅁ')]
[' ']
[('ㅂ', 'ㅕ', 'ㄴ')]
[' ']
[('ㄷ', 'ㅏ', 'ㅈ'), ('ㅆ', 'ㅣ', '')]
[' ']
[('ㄲ', 'ㅏ', 'ㄱ')]
[('ㄲ', 